In [1]:
#pip install --upgrade tomotopy
#pip install reshape
#pip install gensim

import re
import numpy as np
import tomotopy as tp
from kiwipiepy import Kiwi, Option
from pyvis.network import Network

In [2]:
kiwi = Kiwi()

In [3]:
Kiwi(num_workers=-0, model_path='./', options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH)

In [4]:
try:
    corpus = tp.utils.Corpus.load('k.cps')
except IOError:
    kiwi = Kiwi()
    kiwi.add_user_word('전달력', tag='NNG')
    kiwi.add_user_word('영상미', tag='NNG')
    kiwi.add_user_word('영상기법', tag='NNG')
    kiwi.add_user_word('영상구도', tag='NNG')
    kiwi.add_user_word('배경음악', tag='NNG')
    kiwi.add_user_word('구독자수', tag='NNG')   
    kiwi.add_user_word('레고프렌즈', tag='NNG')
    kiwi.add_user_word('딱히', tag='MAG')
    kiwi.add_user_word('교육', tag='NNP')
    kiwi.add_user_word('방송', tag='NNP')
    kiwi.add_user_word('브이로그', tag='NNP')
    kiwi.add_user_word('생일', tag='NNP')
    kiwi.add_user_word('날', tag='NNP')
    kiwi.add_user_word('전반적', tag='NNP')
    kiwi.add_user_word('조회수', tag='NNP')
    kiwi.add_user_word('매니아층', tag='NNP')
    kiwi.add_user_word('생방송', tag='NNP')
    kiwi.add_user_word('먹방', tag='NNP')
    kiwi.add_user_word('첫영상', tag='NNP')
    kiwi.add_user_word('배경음악', tag='NNP')
    kiwi.add_user_word('맛집', tag='NNP')
    kiwi.add_user_word('재건축', tag='NNP')
    kiwi.add_user_word('언박싱', tag='NNP')    
    kiwi.prepare()
    pat_tag = re.compile('NN[GP]|V[VA]|MAG|MM') 

In [5]:
    def tokenizer(raw, user_data):
        res, _ = user_data()[0]
        for w, tag, start, l in res:
            if not pat_tag.match(tag) or len(w) <= 1: continue
            yield w + ('다' if tag.startswith('V') else ''), start, l

In [6]:
    corpus = tp.utils.Corpus(
        tokenizer=tokenizer
    )
    
    corpus.process((line, kiwi.async_analyze(line)) for line in open('for text mining_final_210309_copy2.txt', encoding='UTF-8'))
    corpus.save('data_210309.cps')

case1. tw기본값 (tw=tp.TermWeight.ONE)으로 세팅했을 때!

In [7]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.ONE, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
mdl.train(0)

case2. tw에 PMI로 가중치 부여했을 때

In [7]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
mdl.train(0)

In [8]:
#num_beta_sample은 만 개 이상이면 1~5, 수 천개라면 최소 10, 수 백개에 불과하면 20이상으로 키우기
mdl.num_beta_sample = 10
print('Num docs:{}, Total Words:{}'.format(
    len(mdl.docs), mdl.num_words
))

Num docs:147, Total Words:242


In [10]:
#깁스샘플링
for i in range(0, 500, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)

Iteration: 0000, LL per word: -41.28
Iteration: 0020, LL per word: -41.41
Iteration: 0040, LL per word: -42.98
Iteration: 0060, LL per word: -43.72
Iteration: 0080, LL per word: -44.0
Iteration: 0100, LL per word: -45.66
Iteration: 0120, LL per word: -44.13
Iteration: 0140, LL per word: -43.12
Iteration: 0160, LL per word: -44.78
Iteration: 0180, LL per word: -45.07
Iteration: 0200, LL per word: -44.35
Iteration: 0220, LL per word: -44.11
Iteration: 0240, LL per word: -44.91
Iteration: 0260, LL per word: -45.55
Iteration: 0280, LL per word: -45.75
Iteration: 0300, LL per word: -45.98
Iteration: 0320, LL per word: -46.15
Iteration: 0340, LL per word: -46.45
Iteration: 0360, LL per word: -48.85
Iteration: 0380, LL per word: -46.59
Iteration: 0400, LL per word: -45.47
Iteration: 0420, LL per word: -45.26
Iteration: 0440, LL per word: -46.0
Iteration: 0460, LL per word: -47.96
Iteration: 0480, LL per word: -48.37


In [11]:
print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))

Iteration: 0480, LL per word: -46.62


In [12]:
mdl.summary()

#Words: CTM 분석 후 추출된 문서들 전체의 단어의 개수 (문서 당 같은 단어가 2개 있을 수도 있고 중복 가능)
#Total Vocabs: 작은 빈도의 단어들을 제거한 뒤 남은 토픽모델링 후 어휘의 개수
#Used_Vocabs: 모델에 실제로 사용된 어휘만을 포함하는 dictionary 타입의 어휘 사전
#prior_cov : 토픽의 사전분포인 로지스틱 정규분포의 공분산 행렬
#prior_mean : 토픽의 사전분포인 로지스틱 정규분포의 평균 벡터
#k : 토픽의 개수, 1 ~ 32767 범위의 정수.
#alpha는 CTM 모형의 모수 
#eta는 모수 
#num.iterations는 반복 횟수로 z를 몇 번 추출하느냐


<Basic Info>
| CTModel (current version: 0.10.2)
| 147 docs, 242 words
| Total Vocabs: 268, Used Vocabs: 77
| Entropy of words: -4.23226
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 600, Burn-in steps: 0
| Optimization Interval: 2
| Log-likelihood per word: -46.61535
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 2 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 20 (the number of topics between 1 ~ 32767)
| smoothing_alpha: 0.1 (small smoothing value for preventing topic counts to be zero)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2948873800 (random seed)
| trained in version 0.10.2
|
<Parameters>
| prior_mean (Prior mean of Logit-normal for the per-document topic distributions)
|  [-3.7793534 -3.7121224 -3.6036818 -3.6488166 -2.9888606 -3.6616118
|   -3.4510443 -3.45036   -3.1387978 -3.1192563 -2.908404  -2.8785129
|   -3.1922047 -3.32

In [13]:
mdl.used_vocabs   

#Used Vocabs 의미 >> 78개
#242 words >> k(20)에 속한 총 단어의 수 (84개가 중복되어서 들어감)  == 총 단어의 빈도 수

['브이로그', '일상', '게임', '편집', '좋아하다', '사람', '재미있다', '정보', '주제', '여행', '만들다', '자유', '소통', '공유', '많이', '모습', '강아지', '첫영상', '활동', '도움', '생활', '라이브', '추억', '운동', '취미', '교육', '귀엽다', '패션', '분야', '대하다', '가장', '처음', '시청자', '실시간', '제작', '즐거움', '음악', '관심', '내용', '콘텐츠', '먹방', '업로드', '촬영', '자연', '영상미', '보이다', '고양이', '수업', '아이', '직장', '즐기다', '함께', '노력', '직접', '연주', '노래', '지식', '전달', '기존', '리뷰', '위하다', '방송', '사회', '경험', '기록', '어리다', '공감', '나오다', '가족', '요리', '배경음악', '신경', '요소', '제품', '비교', '올리다', '사용']

In [14]:
#Visualize the result

g = Network(width=800, height=800, font_color="#333")
correl = mdl.get_correlations().reshape([-1])

In [15]:
correl.sort()

In [16]:
# 상관계수 상위 10%만 간선으로 잇고
top_tenth = mdl.k * (mdl.k - 1) // 10
top_tenth = correl[-mdl.k - top_tenth]

In [17]:
topic_counts = mdl.get_count_by_topics()

In [18]:
print(topic_counts)

#합하면 242 (242words) >> 빈도수 242

[11 13 10 13 15  7  6 10 15 18 11 15 11 13  8  8 15 15 17 11]


In [19]:
for k in range(mdl.k):
    label = "#{}".format(k)
    title= ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=8))
    print('Topic', label, title)
    label += '\n' + ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=3))
    g.add_node(k, label=label, title=title, shape='ellipse', value=float(topic_counts[k]))
    for l, correlation in zip(range(k - 1), mdl.get_correlations(k)):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))

Topic #0 재미있다 영상미 자연 요소 좋아하다 사람 편집 정보
Topic #1 편집 기존 신경 배경음악 요소 사람 재미있다 정보
Topic #2 사용 보이다 요리 경험 노력 수업 전달 지식
Topic #3 자유 운동 관심 위하다 사람 좋아하다 재미있다 정보
Topic #4 브이로그 취미 사회 지식 일상 사람 재미있다 정보
Topic #5 먹방 처음 기록 전달 좋아하다 사람 재미있다 정보
Topic #6 직접 연주 노래 지식 좋아하다 사람 재미있다 정보
Topic #7 게임 리뷰 수업 요소 좋아하다 사람 재미있다 정보
Topic #8 많이 도움 어리다 가장 올리다 사람 재미있다 정보
Topic #9 여행 추억 즐거움 패션 고양이 함께 재미있다 정보
Topic #10 사람 라이브 방송 편집 좋아하다 게임 재미있다 정보
Topic #11 좋아하다 정보 모습 즐거움 지식 사람 재미있다 편집
Topic #12 음악 공감 즐기다 비교 제품 전달 재미있다 정보
Topic #13 교육 주제 제작 수업 좋아하다 사람 재미있다 정보
Topic #14 활동 공유 게임 편집 좋아하다 사람 재미있다 정보
Topic #15 실시간 촬영 직장 소통 좋아하다 사람 재미있다 정보
Topic #16 만들다 첫영상 생활 아이 즐거움 처음 모습 정보
Topic #17 내용 분야 대하다 나오다 전달 지식 재미있다 정보
Topic #18 일상 소통 시청자 업로드 요소 사람 재미있다 정보
Topic #19 강아지 귀엽다 콘텐츠 가족 수업 요소 재미있다 정보


In [20]:
#토픽 topic_id에 속하는 상위 top_n개의 단어와 각각의 확률을 반환.
#반환 타입은 (단어:str, 확률:float) 튜플의 list형
#확률은 현재 토픽 내 각각의 단어들의 발생확률?

topic_words = mdl.get_topic_words(k, top_n=50)

In [21]:
import pandas as pd

In [22]:
df_topic_words = pd.DataFrame(topic_words)

In [23]:
#https://dandyrilla.github.io/2017-08-12/pandas-10min/

df_topic_words

,0,1
0,강아지,0.337762
1,귀엽다,0.268085
2,콘텐츠,0.193462
3,가족,0.184431
4,수업,0.000223
5,요소,0.000223
6,재미있다,0.000223
7,정보,0.000223
8,주제,0.000223
9,여행,0.000223


In [24]:
df_topic_words.describe()

,1
count,50.000000
mean,0.019880
std,0.069640
min,0.000223
25%,0.000223
50%,0.000223
75%,0.000223
max,0.337762


In [25]:
topic_words

[('강아지', 0.3377619981765747),
 ('귀엽다', 0.26808470487594604),
 ('콘텐츠', 0.19346237182617188),
 ('가족', 0.18443121016025543),
 ('수업', 0.00022274623916018754),
 ('요소', 0.00022274094226304442),
 ('재미있다', 0.00022273563081398606),
 ('정보', 0.00022273563081398606),
 ('주제', 0.00022273563081398606),
 ('여행', 0.00022273563081398606),
 ('만들다', 0.00022273563081398606),
 ('자유', 0.00022273563081398606),
 ('소통', 0.00022273563081398606),
 ('공유', 0.00022273563081398606),
 ('많이', 0.00022273563081398606),
 ('모습', 0.00022273563081398606),
 ('사람', 0.00022273563081398606),
 ('첫영상', 0.00022273563081398606),
 ('활동', 0.00022273563081398606),
 ('도움', 0.00022273563081398606),
 ('생활', 0.00022273563081398606),
 ('라이브', 0.00022273563081398606),
 ('추억', 0.00022273563081398606),
 ('운동', 0.00022273563081398606),
 ('취미', 0.00022273563081398606),
 ('교육', 0.00022273563081398606),
 ('좋아하다', 0.00022273563081398606),
 ('패션', 0.00022273563081398606),
 ('분야', 0.00022273563081398606),
 ('대하다', 0.00022273563081398606),
 ('가장', 0.00

In [26]:
g.barnes_hut(gravity=-1000, spring_length=20)

In [27]:
g.show_buttons()
g.show("topic_network6.html")

In [28]:
import pyLDAvis
import pyLDAvis.gensim

/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-no

In [29]:
pyLDAvis.enable_notebook()

In [30]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])

In [31]:
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])

In [32]:
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])

In [33]:
vocab = list(mdl.used_vocabs)

In [34]:
term_frequency = mdl.used_vocab_freq

In [35]:
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims= True)

In [36]:
prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
)

/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


ValidationError: 
 * Not all rows (distributions) in doc_topic_dists sum to 1.